<a href="https://colab.research.google.com/github/lg960214/DACON_Seoul_6/blob/feat%2Flhg/Geo_PipeLine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopandas #geopandas
!pip install xlrd==1.2.0 #xls 읽기 위해

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#필수 패키지 임포트
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from tqdm import tqdm
import os

In [3]:
rootdir = '/content/drive/MyDrive/DACON6/지하철 이용 정보/교통카드 통계자료' #승하차 데이터 루트 경로

sub_dirs = {} #sub_dirs[year][month]로 해당 월의 xls 파일경로를 가져올 수 있도록 딕셔너리 만듦.
for year in os.listdir(rootdir):
  sub_sub_dirs = {}
  for month in os.listdir(os.path.join(rootdir, year)):
    sub_sub_dirs[int(month.split(' ')[1][:2])] = os.path.join(rootdir, year, month)
  sub_dirs[int(year)] = sub_sub_dirs

In [6]:
df = pd.read_excel(sub_dirs[2022][1])
df

,사용월,노선ID,노선번호,노선명,버스정류장ID,버스정류장명,승차승객수,하차승객수,작업일시
0,2022-01,11110897,0017,0017번(청암동~이촌동),1,청암자이아파트,453,12,2022-02-03 08:49:57
1,2022-01,11110897,0017,0017번(청암동~이촌동),2,청암동강변삼성아파트,"1,328",43,2022-02-03 08:49:57
2,2022-01,11110897,0017,0017번(청암동~이촌동),3,청심경로당,"2,420",45,2022-02-03 08:49:57
3,2022-01,11110897,0017,0017번(청암동~이촌동),4,원효2동주민센터,"4,196",95,2022-02-03 08:49:57
4,2022-01,11110897,0017,0017번(청암동~이촌동),5,산천동,"5,255",206,2022-02-03 08:49:57
...,...,...,...,...,...,...,...,...,...
39154,2022-01,11110729,중랑02,중랑02(진로아파트~한신아파트),22,서일대정문,108,"2,460",2022-02-03 08:49:57
39155,2022-01,11110729,중랑02,중랑02(진로아파트~한신아파트),23,새마을금고,393,"5,237",2022-02-03 08:49:57
39156,2022-01,11110729,중랑02,중랑02(진로아파트~한신아파트),24,공판장,31,"6,156",2022-02-03 08:49:57
39157,2022-01,11110729,중랑02,중랑02(진로아파트~한신아파트),25,진주빌라.바다약국,26,"9,051",2022-02-03 08:49:57


In [ ]:
dir_shp = '/content/drive/MyDrive/DACON6/서울시 지리 정보/통계지역경계/행정구역.shp' #.shp 파일 경로(서울시 행정구역 단위 경계 데이터)
seoul_shp = gpd.read_file(dir_shp)